In [11]:
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import nltk
nltk.download('stopwords')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from string import punctuation
import re
from collections import Counter
from nltk import ngrams
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [42]:
import numpy as np
from sklearn.datasets import fetch_20newsgroups

train = fetch_20newsgroups(subset='train')
test = fetch_20newsgroups(subset='test')


In [43]:
ex = 1000
print(train["data"][ex])

From: dabl2@nlm.nih.gov (Don A.B. Lindbergh)
Subject: Diamond SS24X, Win 3.1, Mouse cursor
Organization: National Library of Medicine
Lines: 10


Anybody seen mouse cursor distortion running the Diamond 1024x768x256 driver?
Sorry, don't know the version of the driver (no indication in the menus) but it's a recently
delivered Gateway system.  Am going to try the latest drivers from Diamond BBS but wondered
if anyone else had seen this.

post or email

--Don Lindbergh
dabl2@lhc.nlm.nih.gov



In [44]:
from keras.preprocessing.text import Tokenizer

max_nb = 20000

df_train = train["data"]
df_test = test["data"]

# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(nb_words=max_nb, char_level=False)
tokenizer.fit_on_texts(df_train)
sequences = tokenizer.texts_to_sequences(df_train)
sequences_test = tokenizer.texts_to_sequences(df_test)

/usr/local/lib/python3.8/dist-packages/keras/preprocessing/text.py:234: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [46]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlength = 1000
#pad sequence
x_train = pad_sequences(sequences, maxlen=maxlength)
x_test = pad_sequences(sequences_test, maxlen=maxlength)
print('Shape of data tensor:', x_train.shape)
print('Shape of data test tensor:', x_test.shape)

Shape of data tensor: (11314, 1000)
Shape of data test tensor: (7532, 1000)


In [47]:
from keras.utils.np_utils import to_categorical

y_train = train["target"]
y_test = test["target"]

y_train = to_categorical(np.asarray(y_train))
print('Shape of label tensor:', y_train.shape)

Shape of label tensor: (11314, 20)


In [48]:
embedding_dim = 50

In [49]:
sequence_input = Input(shape=(maxlength,), dtype='int32')

embedding_layer = Embedding(max_nb, embedding_dim,
                            input_length=maxlength,
                            trainable=True)
embedded_sequences = embedding_layer(sequence_input)

In [54]:
from keras.layers import LSTM, Conv1D, MaxPooling1D
from keras.models import Model


sequence_input = Input(shape=(maxlength,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# 1D convolution with 64 output channels
x = Conv1D(64, 5)(embedded_sequences)
#divide by 5 
x = MaxPooling1D(5)(x)
x = Conv1D(64, 5)(x)
x = MaxPooling1D(5)(x)
# LSTM
x = LSTM(64)(x)
predictions = Dense(20, activation='softmax')(x)

model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [55]:
from keras.layers import Conv1D, MaxPooling1D, Flatten

sequence_input = Input(shape=(maxlength,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)

# A 1D convolution with 128 output channels
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
# A 1D convolution with 64 output channels
x = Conv1D(64, 5, activation='relu')(x)
# MaxPool divides the length of the sequence by 5
x = MaxPooling1D(5)(x)
x = Flatten()(x)

predictions = Dense(20, activation='softmax')(x)

model = Model(sequence_input, predictions)
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

high batch -> can cause exhausted resource error that can cause by running out of memory.
low batch -> slower training speed

In [41]:
model.fit(x_train, y_train, validation_split=0.1,
          epochs=10, batch_size=128, verbose=2)

Epoch 1/10
80/80 - 80s - loss: 2.9391 - acc: 0.0757 - val_loss: 2.8160 - val_acc: 0.1131 - 80s/epoch - 1s/step
Epoch 2/10
80/80 - 80s - loss: 2.3979 - acc: 0.1701 - val_loss: 2.1768 - val_acc: 0.2173 - 80s/epoch - 997ms/step
Epoch 3/10
80/80 - 76s - loss: 1.8478 - acc: 0.3360 - val_loss: 1.9270 - val_acc: 0.3322 - 76s/epoch - 953ms/step
Epoch 4/10
80/80 - 88s - loss: 1.2116 - acc: 0.5857 - val_loss: 1.4626 - val_acc: 0.5415 - 88s/epoch - 1s/step
Epoch 5/10
80/80 - 75s - loss: 0.6195 - acc: 0.7935 - val_loss: 1.3816 - val_acc: 0.6299 - 75s/epoch - 933ms/step
Epoch 6/10
80/80 - 83s - loss: 0.3031 - acc: 0.9135 - val_loss: 1.4549 - val_acc: 0.6643 - 83s/epoch - 1s/step
Epoch 7/10
80/80 - 75s - loss: 0.1614 - acc: 0.9593 - val_loss: 1.6511 - val_acc: 0.6837 - 75s/epoch - 934ms/step
Epoch 8/10
80/80 - 87s - loss: 0.0860 - acc: 0.9837 - val_loss: 1.7508 - val_acc: 0.6952 - 87s/epoch - 1s/step
Epoch 9/10
80/80 - 94s - loss: 0.0583 - acc: 0.9887 - val_loss: 1.9039 - val_acc: 0.6890 - 94s/epoch

In [59]:
num_epochs = 5
early_stop = EarlyStopping(monitor = 'val_loss',
                           patience = 2)

history = model.fit(x_train, y_train, 
                    validation_split=0.1,
                    epochs=10, 
                    batch_size=32,
                    callbacks = [early_stop],
                    verbose = 2)

Epoch 1/10
319/319 - 88s - loss: 0.0056 - acc: 0.9992 - val_loss: 2.2836 - val_acc: 0.7341 - 88s/epoch - 276ms/step
Epoch 2/10
319/319 - 86s - loss: 0.0278 - acc: 0.9943 - val_loss: 2.6787 - val_acc: 0.7120 - 86s/epoch - 269ms/step
Epoch 3/10
319/319 - 85s - loss: 0.0776 - acc: 0.9805 - val_loss: 2.3742 - val_acc: 0.7465 - 85s/epoch - 267ms/step


In [80]:
model.save('lstm')
